In [15]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

In [16]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [17]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [18]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    555     555 

## Models

In [19]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [20]:

model_monmlp <- train(x=TrainData, y=TrainClasses, 
               method = "monmlp") 

** Ensemble 1 
Complex eigenvalues found for method = BFGS 
coefficients for function value 0.3517126  :
  [1] -1.604668e+02  3.513691e+02  4.172245e+00  1.148732e+02  2.917543e+02
  [6] -2.672013e+02 -3.212349e+02 -7.333217e+00  7.331425e+00  9.515256e+01
 [11] -1.233994e+02 -1.286599e+02 -3.257854e+01  8.986861e+01  5.959884e+01
 [16] -5.956861e+01  2.752564e+01  1.417935e+01  3.156411e+01 -4.593777e+01
 [21] -7.543612e+01  7.548475e+01 -3.597786e+01 -1.306577e+02  8.191117e+01
 [26]  2.929339e+01  6.032347e+01  7.665152e+01 -7.665560e+01  2.862276e+01
 [31] -2.868869e+01 -1.242372e+02  1.242250e+02 -3.444032e+01  2.138840e+02
 [36]  1.086732e+02 -1.574909e+02 -1.056003e+02  1.148997e+02 -1.148651e+02
 [41] -7.519549e+01  1.030768e+02 -9.395101e+00  2.318222e+00 -1.806994e+02
 [46]  3.191822e+02 -2.624459e+02 -3.220003e+01 -1.307043e+02 -7.041750e+01
 [51] -7.307386e+01 -2.894737e+02 -2.293909e+00 -3.340218e+01  1.779893e+01
 [56]  2.931628e+01 -2.484471e+01 -6.260537e+01  5.012775e+

In [21]:

model_mlpW <- train(x=TrainData, y=TrainClasses, 
               method = "mlpWeightDecay") 

## Results

In [22]:
print(model_monmlp)
plot(model_monmlp,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Monotone Multi-Layer Perceptron Neural Network 

1110 samples
 174 predictor
   2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 1110, 1110, 1110, 1110, 1110, 1110, ... 
Resampling results across tuning parameters:

  hidden1  Accuracy   Kappa    
  1        0.6928342  0.3849152
  3        0.6891352  0.3781023
  5        0.6742482  0.3480124

Tuning parameter 'n.ensemble' was held constant at a value of 1
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were hidden1 = 1 and n.ensemble = 1.


In [23]:
print(model_mlpW)
plot(model_mlpW,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Multi-Layer Perceptron 

1110 samples
 174 predictor
   2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 1110, 1110, 1110, 1110, 1110, 1110, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.7252887  0.4501490
  1     1e-04  0.7233262  0.4449173
  1     1e-01  0.4932343  0.0000000
  3     0e+00  0.7217545  0.4437298
  3     1e-04  0.7178852  0.4361627
  3     1e-01  0.4971965  0.0000000
  5     0e+00  0.7209715  0.4418228
  5     1e-04  0.7348020  0.4688981
  5     1e-01  0.4962487  0.0000000

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 1e-04.


In [24]:
# collect resamples
results <- resamples(list(monmlp=model_monmlp,  mlpWeightDecay=model_mlpW))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: monmlp, mlpWeightDecay 
Number of resamples: 25 

Accuracy 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.6507538 0.6775000 0.6896552 0.6928342 0.7039801 0.7404762    0
mlpWeightDecay 0.7017115 0.7206983 0.7368421 0.7348020 0.7481108 0.7657005    0

Kappa 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.2917424 0.3581770 0.3753115 0.3849152 0.4060914 0.4808111    0
mlpWeightDecay 0.3971924 0.4394223 0.4692541 0.4688981 0.4980529 0.5326109    0


## Prediction

In [25]:
pred_mlpW <- predict(model_mlpW,TestData)
pred_mlpW.prob <- predict(model_mlpW,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_mlpW) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     110      61
   NotFast   67     238
                                          
               Accuracy : 0.7311          
                 95% CI : (0.6889, 0.7704)
    No Information Rate : 0.6282          
    P-Value [Acc > NIR] : 1.274e-06       
                                          
                  Kappa : 0.4204          
                                          
 Mcnemar's Test P-Value : 0.6585          
                                          
            Sensitivity : 0.6215          
            Specificity : 0.7960          
         Pos Pred Value : 0.6433          
         Neg Pred Value : 0.7803          
             Prevalence : 0.3718          
         Detection Rate : 0.2311          
   Detection Prevalence : 0.3592          
      Balanced Accuracy : 0.7087          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.6214689            0.7959866            0.6432749 
      Neg Pred Value            Precision               Recall 
           0.7803279            0.6432749            0.6214689 
                  F1           Prevalence       Detection Rate 
           0.6321839            0.3718487            0.2310924 
Detection Prevalence    Balanced Accuracy 
           0.3592437            0.7087278

In [26]:
roc_obj <- multiclass.roc(TestClasses, as.numeric(pred_mlpW))
roc_obj


Setting direction: controls < cases




Call:
multiclass.roc.default(response = TestClasses, predictor = as.numeric(pred_mlpW))

Data: as.numeric(pred_mlpW) with 2 levels of TestClasses: fast, NotFast.
Multi-class area under the curve: 0.7118

In [27]:
pred_mlpW.prob

,fast,NotFast
NA.575,0.84036553,0.1596344
NA.687,0.11576660,0.8842334
NA.692,0.88499033,0.1150097
NA.929,0.40044004,0.5995600
NA.986,0.51457071,0.4854293
NA.1005,0.85305828,0.1469418
NA.1048,0.21566486,0.7843351
NA.2048,0.63142103,0.3685789
NA.2398,0.01747282,0.9825272
NA.2431,0.22817938,0.7718207
